In [ ]:
############## LIBRERIAS ##############
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.mask import mask
import subprocess
import numpy as np
import scipy.ndimage
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import random
import shutil

In [ ]:
########## ALINEACIÓN DE IMÁGENES CON MÁSCARAS, AJUSTE PARA QUE LA IMAGEN QUEDE CENTRADA EN LA MÁSCARA #########
# Este script alinea imágenes de remociones con sus respectivas máscaras, asegurando que las imágenes
# estén correctamente centradas y ajustadas a la resolución de las máscaras.
# Directorios de entrada y salida
input_remociones_dir = r"E:\Memoria\CNN\Originales\TIFF\RM"         # Imágenes originales de remociones
input_masks_dir = r"E:\Memoria\CNN\Originales\TIFF\ground_truth"   # Máscaras originales
output_remociones_dir = r"E:\Memoria\CNN\pruebasMartes\Aligned\RM"  # Salida imágenes alineadas
output_masks_dir = r"E:\Memoria\CNN\pruebasMartes\Aligned\Masks"    # Salida máscaras alineadas

# Crear carpetas de salida
os.makedirs(output_remociones_dir, exist_ok=True)
os.makedirs(output_masks_dir, exist_ok=True)

# Función para alinear imágenes con las máscaras como referencia
def align_to_mask(image_path, mask_path):
    with rasterio.open(mask_path) as mask_src:
        with rasterio.open(image_path) as img_src:
            # Verificar que ambas tengan el mismo CRS
            if img_src.crs != mask_src.crs:
                raise ValueError(f"CRS no coincide entre {image_path} y {mask_path}.")
            
            # Reproyectar la imagen al tamaño y resolución de la máscara
            img_aligned = img_src.read(
                out_shape=(img_src.count, mask_src.height, mask_src.width),
                resampling=Resampling.lanczos  # Lanczos para imágenes
            )
            
            # Ajustar perfil para que coincida con la máscara
            profile = mask_src.profile
            profile.update({
                "count": img_src.count,  # Mantener la cantidad de bandas original
                "driver": "GTiff",       # Guardar en formato TIFF
            })

            return img_aligned, profile

# Procesar imágenes y máscaras
print("Alineando imágenes a sus máscaras...")
for filename in os.listdir(input_masks_dir):
    if filename.endswith("_ground_truth.tif"):
        base_name = filename.replace("_ground_truth.tif", "")
        image_filename = f"{base_name}.tif"
        
        # Rutas completas
        mask_path = os.path.join(input_masks_dir, filename)
        image_path = os.path.join(input_remociones_dir, image_filename)
        
        if os.path.exists(image_path):
            try:
                # Alinear imagen usando la máscara como referencia
                img_aligned, profile = align_to_mask(image_path, mask_path)

                # Guardar la imagen alineada
                output_image_path = os.path.join(output_remociones_dir, image_filename)
                with rasterio.open(output_image_path, "w", **profile) as dst:
                    dst.write(img_aligned)

                # Copiar la máscara al directorio de salida sin cambios
                output_mask_path = os.path.join(output_masks_dir, filename)
                with rasterio.open(mask_path) as src:
                    mask_data = src.read()
                    with rasterio.open(output_mask_path, "w", **src.profile) as dst:
                        dst.write(mask_data)

                print(f"Alineado y guardado: {image_filename} y {filename}")
            except Exception as e:
                print(f"Error procesando {image_filename} y {filename}: {e}")
        else:
            print(f"Advertencia: No se encontró la imagen correspondiente para {filename}")

print("¡Alineación completada correctamente!")

In [ ]:
####### RECORTE DE RASTERS CON IMÁGENES DE MUESTRA ##########
# Este script recorta rasters morfométricos utilizando imágenes de muestra como referencia.
# Directorios
morfometricos_dir = r"E:\Memoria\GIS\Predictores"  # Carpeta con los rasters
muestras_dir = r"E:\Memoria\CNN\pruebasMartes\OG_Fixed\TIF_fixed\RM"  # Carpeta con imágenes de muestra (TIFF)
output_dir = r"E:\Memoria\CNN\PreProcesamiento\rasters_recortados2"  # Carpeta de salida para rasters recortados

# Crear carpeta de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Lista reducida a 2 rasters:
rasters_a_recortar = [
    "Slope_grados.sdat",
    "Aspect.sdat",
    "Convexity.sdat",
    "Landforms.sdat",
    "Terrain Ruggedness Index (TRI).sdat",
    "Topographic Position Index.sdat",
    "Vector Terrain Ruggedness (VRM).sdat",
    "Radar2020.tif"
]

# Obtener todas las imágenes de muestra
muestras_tiff = [f for f in os.listdir(muestras_dir) if f.endswith(".tif")]

for img_name in muestras_tiff:
    img_path = os.path.join(muestras_dir, img_name)

    with rasterio.open(img_path) as img_src:
        bounds = img_src.bounds  # Obtener límites geográficos de la imagen de muestra
        geom = [{
            "type": "Polygon",
            "coordinates": [[
                [bounds.left, bounds.bottom],
                [bounds.left, bounds.top],
                [bounds.right, bounds.top],
                [bounds.right, bounds.bottom],
                [bounds.left, bounds.bottom]
            ]]
        }]
    
    print(f"Procesando imagen {img_name}...")

    for raster_name in rasters_a_recortar:
        raster_path = os.path.join(morfometricos_dir, raster_name)

        # Verificar si el archivo existe
        if not os.path.exists(raster_path):
            print(f"⚠ El raster {raster_name} no fue encontrado.")
            continue

        # Si es un archivo .sdat, convertirlo a GeoTIFF temporalmente
        if raster_name.endswith(".sdat"):
            temp_tif = raster_path.replace(".sdat", ".tif")
            if not os.path.exists(temp_tif):  # Convertir solo si no existe el GeoTIFF
                print(f"🔄 Convirtiendo {raster_name} a GeoTIFF...")
                subprocess.run(["gdal_translate", "-of", "GTiff", raster_path, temp_tif], check=True)
            raster_path = temp_tif  # Usamos el archivo convertido

        # Recortar el raster
        with rasterio.open(raster_path) as src:
            try:
                out_image, out_transform = mask(src, geom, crop=True)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })

                # 🔹 **Nombre de salida = RasterOriginal_Imagen.tif**
                raster_base_name = raster_name.replace(".sdat", "").replace(".tif", "")  # Sin extensión
                img_base_name = img_name.replace(".tif", "")  # Sin extensión
                output_raster_path = os.path.join(output_dir, f"{raster_base_name}_{img_base_name}.tif")

                with rasterio.open(output_raster_path, "w", **out_meta) as dest:
                    dest.write(out_image)

                print(f"Raster recortado guardado: {output_raster_path}")

            except Exception as e:
                print(f"Error al recortar {raster_name} para {img_name}: {e}")

print("¡Todos los recortes han finalizado!")

In [ ]:
############# FUSIÓN DE IMÁGENES RGB CON RASTERS MORFOMÉTRICOS EN FORMATO NUMPY #############
# Este script fusiona imágenes RGB con rasters morfométricos, guardando el resultado en formato NumPy.
# Directorios
imagenes_dir = r"E:\Memoria\CNN\Originales\TIFF\RM"
morfometria_dir = r"E:\Memoria\CNN\PreProcesamiento\rasters_recortados2"
output_dir = r"E:\Memoria\CNN\PreProcesamiento\fusionadas_numpy2"

# Crear carpeta de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Obtener lista de imágenes TIFF
imagenes_tiff = sorted([f for f in os.listdir(imagenes_dir) if f.endswith(".tif")])
morfometricos_tiff = sorted([f for f in os.listdir(morfometria_dir) if f.endswith(".tif")])

# Diccionario para guardar nombres de canales por imagen
canales_por_imagen = {}

# Procesar cada imagen
for img_name in imagenes_tiff:
    img_path = os.path.join(imagenes_dir, img_name)
    img_base_name = os.path.splitext(img_name)[0]  # Ejemplo: "car1", "des4", "rm2"

    with rasterio.open(img_path) as src:
        # Cargar imagen RGB
        imagen_base = src.read([1, 2, 3])  # Bandas RGB
        img_size = (src.height, src.width)
        perfil_imagen = src.profile  # Guardar metadatos
    
    # Buscar rasters que correspondan al img_base_name actual
    rasters_asociados = []
    for f in morfometricos_tiff:
        # Extraer el image_id del archivo morfométrico (última parte después de '_')
        morf_base = os.path.splitext(f)[0]
        morf_image_id = morf_base.split('_')[-1]
        if morf_image_id == img_base_name:
            rasters_asociados.append(os.path.join(morfometria_dir, f))
    
    if not rasters_asociados:
        print(f"⚠ No se encontraron rasters para {img_name}. Saltando...")
        continue

    print(f"Fusionando {len(rasters_asociados)} rasters con {img_name}...")

    # Lista para registrar el nombre de cada canal
    nombres_canales = ["Red", "Green", "Blue"]

    # Cargar y redimensionar los rasters morfométricos
    rasters_morfometricos = []
    for raster_path in rasters_asociados:
        with rasterio.open(raster_path) as src:
            raster_resampled = src.read(
                1, out_shape=img_size, resampling=Resampling.bilinear
            )

            # 🛠 Corrección de valores NODATA
            nodata_value = src.nodata
            if nodata_value is not None:
                raster_resampled = np.where(raster_resampled == nodata_value, np.nan, raster_resampled)

            # Opcional: reemplazar NaN con la media de valores válidos
            if np.isnan(raster_resampled).any():
                media_valida = np.nanmean(raster_resampled)
                raster_resampled = np.where(np.isnan(raster_resampled), media_valida, raster_resampled)

            rasters_morfometricos.append(raster_resampled)
            nombres_canales.append(os.path.basename(raster_path))  # Guardar nombre del raster

    # Convertir la imagen base a formato (H, W, C)
    imagen_base = np.moveaxis(imagen_base, 0, -1)  # De (3, H, W) a (H, W, 3)
    
    # Apilar los rasgos morfométricos
    rasters_morfometricos = np.stack(rasters_morfometricos, axis=-1)  # (H, W, N)
    
    # Fusionar imagen con rasgos
    imagen_fusionada = np.concatenate([imagen_base, rasters_morfometricos], axis=-1)  # (H, W, C+N)
    
    # Guardar la imagen fusionada en formato NumPy
    output_path = os.path.join(output_dir, f"{img_base_name}_fusionada.npy")
    np.save(output_path, imagen_fusionada)

    # Guardar nombres de los canales en un diccionario
    canales_por_imagen[img_base_name] = nombres_canales

    print(f"Imagen fusionada guardada en: {output_path}")

# Guardar los nombres de los canales en un archivo de texto
canales_output_path = os.path.join(output_dir, "nombres_canales.txt")
with open(canales_output_path, "w") as f:
    for img, canales in canales_por_imagen.items():
        f.write(f"{img}: {', '.join(canales)}\n")

print(f"Archivo con nombres de canales guardado en: {canales_output_path}")
print("¡Proceso de fusión completado!")


In [ ]:
############## REDIMENSIONAMIENTO DE IMÁGENES FUSIONADAS A 512x512 ################
# Este script redimensiona las imágenes fusionadas a un tamaño de 512x512 píxeles.
# Directorios
fusionadas_dir = r"E:\Memoria\CNN\fusionadas_sin_redimensionar"  # Carpeta con imágenes fusionadas
output_dir = r"E:\Memoria\CNN\fusionadas_redimensionadas"  # Carpeta de salida

# Crear carpeta de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Obtener lista de archivos numpy fusionados
fusionadas_files = sorted([f for f in os.listdir(fusionadas_dir) if f.endswith(".npy")])

# Tamaño deseado
target_size = (512, 512)

for file_name in fusionadas_files:
    file_path = os.path.join(fusionadas_dir, file_name)

    # Cargar la imagen fusionada
    fused_img = np.load(file_path)

    # Obtener dimensiones actuales
    h, w, c = fused_img.shape

    # Redimensionar cada canal usando SciPy
    resized_bands = [
        scipy.ndimage.zoom(fused_img[..., i], (target_size[0] / h, target_size[1] / w), order=1)
        for i in range(c)
    ]

    # Apilar bandas redimensionadas
    resized_fused_img = np.stack(resized_bands, axis=-1)

    # Guardar la imagen redimensionada
    output_path = os.path.join(output_dir, file_name)
    np.save(output_path, resized_fused_img)

    print(f"Redimensionada y guardada: {output_path}")

print("¡Todas las imágenes fusionadas han sido redimensionadas correctamente a 512x512!")

In [ ]:
############## VERIFICACIÓN DE IMÁGENES FUSIONADAS REDIMENSIONADAS ################
# Este script verifica las imágenes fusionadas y redimensionadas, asegurando que tengan las dimensiones correctas.
# Directorio con las imágenes fusionadas y redimensionadas
redimensionadas_dir = r"E:\Memoria\CNN\fusionadas_redimensionadas"

# Obtener archivos numpy redimensionados
redimensionadas_files = sorted([f for f in os.listdir(redimensionadas_dir) if f.endswith(".npy")])

if not redimensionadas_files:
    print("❌ No se encontraron archivos .npy en la carpeta de redimensionadas.")
    exit()

# Seleccionar una imagen de muestra
sample_file = os.path.join(redimensionadas_dir, redimensionadas_files[0])
fused_img = np.load(sample_file)

# Verificar dimensiones
print(f"📌 Imagen: {sample_file}")
print(f"Dimensiones: {fused_img.shape}")  # Debería ser (512, 512, 6)

# Separar los canales correctamente
rgb = fused_img[..., :3] / 255.0  # Normalizar RGB a [0,1]
slope = fused_img[..., 3]
tri = fused_img[..., 4]
vrm = fused_img[..., 5]  # Cambio de TPI a VRM

# Normalizar los canales adicionales para mejorar la visualización
def normalize(array):
    array[array < -10000] = np.nan  # Reemplazar valores inválidos (-99999) con NaN
    min_val, max_val = np.nanmin(array), np.nanmax(array)  # Ignorar NaN en cálculos
    return (array - min_val) / (max_val - min_val) if max_val > min_val else array

slope_normalized = normalize(slope)
tri_normalized = normalize(tri)
vrm_normalized = normalize(vrm)

# Mostrar los canales de la imagen
plt.figure(figsize=(14, 6))

# Mostrar RGB (normalizado)
plt.subplot(1, 4, 1)
plt.imshow(rgb)
plt.title("RGB Original")

# Mostrar Slope
plt.subplot(1, 4, 2)
plt.imshow(slope_normalized, cmap="terrain")
plt.title("Slope (Normalizado)")

# Mostrar TRI
plt.subplot(1, 4, 3)
plt.imshow(tri_normalized, cmap="terrain")
plt.title("TRI (Normalizado)")

# Mostrar VRM
plt.subplot(1, 4, 4)
plt.imshow(vrm_normalized, cmap="terrain")
plt.title("VRM (Normalizado)")

plt.tight_layout()
plt.show()


In [ ]:
############## REDIMENSIONAMIENTO DE MASCARAS A 512x512 ################
# Este script redimensiona las máscaras a un tamaño de 512x512 píxeles y las guarda en formato NumPy.
# 🔹 Directorios
mascaras_originales_dir = r"E:\Memoria\CNN\pruebasMartes\OG_Fixed\TIF_fixed\Masks"
output_dir = r"E:\Memoria\CNN\masks_redimensionadas"

# Crear carpeta de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Obtener lista de máscaras originales
mascaras_files = sorted([f for f in os.listdir(mascaras_originales_dir) if f.endswith(".tif")])

for mask_name in mascaras_files:
    mask_path = os.path.join(mascaras_originales_dir, mask_name)
    output_path = os.path.join(output_dir, mask_name.replace(".tif", ".npy"))

    # Cargar máscara
    with rasterio.open(mask_path) as src:
        mask = src.read(1)  # Leer la única banda (grayscale)
        mask_resampled = src.read(
            1, out_shape=(512, 512), resampling=Resampling.nearest  # Nearest neighbor para preservar etiquetas
        )

    # Asegurar valores enteros y tipo correcto
    mask_resampled = mask_resampled.astype(np.uint8)

    # Guardar en formato NumPy
    np.save(output_path, mask_resampled)
    print(f"Máscara redimensionada guardada: {output_path}")

print("Todas las máscaras han sido redimensionadas correctamente.")

In [ ]:
############## VERIFICACION DE ALINEACIÓN DE IMÁGENES Y MÁSCARAS ################
# Este script verifica que las imágenes y las máscaras redimensionadas tengan nombres coincidentes
# y dimensiones correctas, y permite visualizar algunas muestras de forma interactiva.
# 🔹 Directorios
imagenes_dir = r"E:\Memoria\CNN\fusionadas_redimensionadas"  # Carpeta con imágenes de 6 canales en formato .npy
mascaras_dir = r"E:\Memoria\CNN\masks_redimensionadas"  # Carpeta con máscaras redimensionadas en formato .npy

# Obtener nombres de archivos
imagenes_files = {os.path.splitext(f)[0]: f for f in os.listdir(imagenes_dir) if f.endswith(".npy")}
mascaras_files = {os.path.splitext(f)[0].replace("_ground_truth", ""): f for f in os.listdir(mascaras_dir) if f.endswith(".npy")}

# **Verificar coincidencias**
imagenes_set = set(imagenes_files.keys())
mascaras_set = set(mascaras_files.keys())

if imagenes_set != mascaras_set:
    print(f"⚠️ ERROR: No coinciden todas las imágenes y máscaras.")
    print(f"Imágenes sin máscara: {imagenes_set - mascaras_set}")
    print(f"Máscaras sin imagen: {mascaras_set - imagenes_set}")
else:
    print("✅ Los nombres de imágenes y máscaras coinciden correctamente.")

# 📌 Seleccionar 10 imágenes aleatorias
muestras = random.sample(list(imagenes_files.keys()), min(10, len(imagenes_files)))  # Evitar error si hay menos de 10

# **Visualización interactiva**
for nombre in muestras:
    imagen_path = os.path.join(imagenes_dir, imagenes_files[nombre])
    mascara_path = os.path.join(mascaras_dir, mascaras_files[nombre])

    # Cargar imagen y máscara
    imagen = np.load(imagen_path)  # Shape esperado: (512, 512, 6)
    mascara = np.load(mascara_path)  # Shape esperado: (512, 512)

    # **Verificar dimensiones**
    if imagen.shape[:2] != mascara.shape:
        print(f"⚠️ Dimensiones diferentes en {nombre}: Imagen {imagen.shape[:2]}, Máscara {mascara.shape}")
    else:
        print(f"✅ Dimensiones correctas en {nombre}")

    # **Normalizar imagen para visualización**
    imagen = imagen.astype(np.float32)  # Asegurar tipo float
    imagen = np.clip(imagen, 0, 255)  # Evitar valores negativos o fuera de rango
    imagen /= imagen.max()  # Normalizar a [0,1] para que matplotlib la muestre bien

    # **Mostrar imágenes una por una**
    fig, axes = plt.subplots(1, 3, figsize=(12, 5))

    # 📌 Mostrar imagen en RGB (bandas 3,2,1)
    axes[0].imshow(imagen[:, :, [3, 2, 1]])  # RGB
    axes[0].set_title(f"Imagen {nombre} (RGB)")
    axes[0].axis("off")

    # 📌 Mostrar máscara en escala de grises
    axes[1].imshow(mascara, cmap="gray")
    axes[1].set_title("Máscara")
    axes[1].axis("off")

    # 📌 Superponer máscara sobre la imagen
    axes[2].imshow(imagen[:, :, [3, 2, 1]])  # Imagen de fondo en RGB
    axes[2].imshow(mascara, cmap="jet", alpha=0.5)  # Superposición
    axes[2].set_title("Superposición")
    axes[2].axis("off")

    plt.tight_layout()
    plt.show()  # 🔹 Aquí el script **se pausa** hasta que cierres la imagen

In [ ]:
############## DIVISIÓN DEL DATASET EN TRAIN, VAL Y TEST ################
# Este script divide el dataset de imágenes fusionadas y máscaras en conjuntos de entrenamiento, validación y prueba.
# 🔹 Rutas de los datos originales
imagenes_dir = r"E:\Memoria\CNN\fusionadas_redimensionadas"  
mascaras_dir = r"E:\Memoria\CNN\masks_redimensionadas"

# 🔹 Rutas de destino para cada conjunto
output_base = r"E:\Memoria\CNN\Dataset_Split_npy"
train_dir = os.path.join(output_base, "train")
val_dir = os.path.join(output_base, "val")
test_dir = os.path.join(output_base, "test")

# 🔹 Crear carpetas si no existen
for subdir in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(subdir, "images"), exist_ok=True)
    os.makedirs(os.path.join(subdir, "masks"), exist_ok=True)

# 🔹 Obtener lista de archivos
imagenes_files = sorted([f for f in os.listdir(imagenes_dir) if f.endswith(".npy")])
mascaras_files = sorted([f for f in os.listdir(mascaras_dir) if f.endswith(".npy")])

# 🔹 Asegurar correspondencia entre imágenes y máscaras
imagenes_dict = {f.replace(".npy", ""): f for f in imagenes_files}
mascaras_dict = {f.replace("_ground_truth.npy", ""): f for f in mascaras_files}

# 🔹 Filtrar solo archivos que tienen su par correspondiente
paired_files = sorted(set(imagenes_dict.keys()) & set(mascaras_dict.keys()))
np.random.shuffle(paired_files)  # Mezclar aleatoriamente

# 🔹 División del dataset (50% train, 20% val, 30% test)
num_total = len(paired_files)
num_train = int(0.5 * num_total)
num_val = int(0.2 * num_total)

train_set = paired_files[:num_train]
val_set = paired_files[num_train:num_train + num_val]
test_set = paired_files[num_train + num_val:]

# 🔹 Función para mover archivos a la carpeta correspondiente
def move_files(file_list, dataset_type):
    img_dst = os.path.join(output_base, dataset_type, "images")
    mask_dst = os.path.join(output_base, dataset_type, "masks")
    
    for name in file_list:
        img_src = os.path.join(imagenes_dir, imagenes_dict[name])
        mask_src = os.path.join(mascaras_dir, mascaras_dict[name])
        
        shutil.move(img_src, os.path.join(img_dst, imagenes_dict[name]))
        shutil.move(mask_src, os.path.join(mask_dst, mascaras_dict[name]))

# 🔹 Mover archivos
move_files(train_set, "train")
move_files(val_set, "val")
move_files(test_set, "test")

print(f"División completada: {num_train} train, {num_val} val, {num_total - num_train - num_val} test.")
